In [ ]:
### Custom user input parameters:
# Would you like log Q/K or affinity (in kcal)?
output_type <- "affinity" # "logQoverK" or "affinity"

### End of custom user input parameters

In [ ]:
setwd("rxn_3o")

In [ ]:
n_files <- length(list.files())/3
sample_files <- paste(1:n_files, ".3o", sep="")

In [ ]:
master_df <- data.frame(sample = character(0), Eh = numeric(0), stringsAsFactors=FALSE)

for(this_file in sample_files){

  # read .3o file as a string
  fileName <- this_file
  extractme <- readChar(fileName, file.info(fileName)$size) # readChar requires filesize, which is obtained from file.info()$size

  # getting Eh in two steps by using sub()
  # 1. trim off everything before Eh
  Eh_trailing <- sub("^.*NBS pH scale\\s+\\S+\\s+", "", extractme)
  # 2. trim off everything after Eh
  this_Eh <- sub("\\s+.*$", "", Eh_trailing)
  this_Eh <- as.numeric(this_Eh) # returns NA if this_Eh is not numeric

  # assign a placeholder value of -999 if Eh is not numeric
  if(is.na(this_Eh)){
    this_Eh <- NA
  }
  this_Eh

  # bind Eh to master data frame
  master_df[nrow(master_df)+1, ] <- rep(NA, ncol(master_df)) # create a new row
  master_df[nrow(master_df), "sample"] <- this_file
  master_df[nrow(master_df), "Eh"] <- this_Eh


  # string to isolate the mineral saturation section:
  front_trim <- "^.*\n\n\n\n           --- Saturation States of Pure Solids ---\n\n       Phase                      Log Q/K    Affinity, kcal\n\n\\s+"

  # check if mineral block exists. If not, skip processing the file:
  if (grepl(front_trim, extractme) == FALSE){
    next # skip to next sample file
  }

  # trim off everything before:
  mineral_trailing <- sub(front_trim, "", extractme)
  # trim off everything after:
  mineral_block <- sub("\n\n.*$", "", mineral_trailing)
  # replace all commas with underscores
  mineral_block <- gsub(",", "_", mineral_block)
  # split into substrings, each representing a separate row in the table
  mineral_block <- strsplit(mineral_block, "\n")




  #create an empty data frame to store results
  df <- data.frame(mineral = character(0),
                   logQoverK = numeric(0),
                   affinity = numeric(0),
                   stringsAsFactors = FALSE)
  # convert into dataframe
  for(this_row in mineral_block[[1]]){
    # remove any leading spaces
    this_row <- sub("^\\s+", "", this_row)
    # trim off everything after mineral name to get this_mineral
    this_mineral <- sub("\\s+.*$", "", this_row)
    # to get LogQ/K, trim off everything before it
    this_LogQoverK_trailing <- sub("^.*[A-Z]\\s+", "", this_row)
    # then trim off everything after it
    this_LogQoverK <- sub("\\s+.*$", "", this_LogQoverK_trailing)
    # to get Affinity, trim off everything before it:
    this_Affinity_trailing <- sub("^.*[A-Z]\\s+\\S+\\s+", "", this_row)
    # then trim off everything after it
    this_Affinity <- sub("\\s+.*$", "", this_Affinity_trailing)
    # create a dataframe with results
    this_df <- data.frame(mineral = this_mineral,
                          logQoverK = as.numeric(this_LogQoverK),
                          affinity = as.numeric(this_Affinity),
                          stringsAsFactors = FALSE)
    # bind results to dataframe
    df <- rbind(df, this_df)
  }
  df

  # add mineral affinities to master_df
  for(this_mineral in df[, "mineral"]){
    # if this_mineral does not have a column in the master_df, add it
    if (!(this_mineral %in% colnames(master_df))){
      master_df <- cbind(master_df, rep(NA, nrow(master_df))) # add extra column
      colnames(master_df)[ncol(master_df)] <- this_mineral # rename added column
    }

    master_df[nrow(master_df), this_mineral] <- df[which(df[, "mineral"] == this_mineral), output_type]
  }

}
# sort mineral columns in alphabetical order
unsorted_mineral_names <- colnames(master_df[, 3:ncol(master_df)])
master_df <- master_df[, c("sample", "Eh", sort(unsorted_mineral_names))]
master_df

setwd("../")
write.csv(master_df, paste("mineral_", output_type,".csv", sep=""))